# Day 19: An Elephant Named Joseph

https://adventofcode.com/2016/day/19

## Part 1

### First solution, a dictionary as a linked list

It works, but it's not fast!

In [14]:
def whiteElephantDict(n):
    neig = {}
    for i in range(1,n):
        neig[i] = (i+1)
    neig[n] = 1
    i = 1
    while len(neig)>1:
        if i in neig.keys():
            ne = neig[i]
            neig[i] = neig[ne]
            del neig[ne]
        i+=1
        if i>n:
            i=1
    return list(neig.keys())[0]

In [16]:
print(whiteElephantDict(3))

3


In [18]:
from timeit import default_timer as timer
start = timer()

print(whiteElephantDict(3014387))

stop = timer()
print("Time elapsed =",int(stop-start),"s")

1834471
Time elapsed = 14 s


## Part 1 

### A better idea!

By observing how the list of playing elfs evolves, I realised I can simply pop elements from it according to a simple algorithm with slightly different behavior according to whether the lenght of the list is odd (the last player takes the gift of the first, that gets eliminated) or even (the last player gets eliminated).

In [3]:
def whiteElephantFast(n):
    # inizialize player list
    playing = [i for i in range(1,n+1)]
    while len(playing)>1:
        if len(playing)%2==1: # odd number of players
            # remove "even" players
            playing = playing[0::2]
            # last player gets gifts from first player, so pop first player
            playing.pop(0)
            # I'm left with a even number of players
        if len(playing)%2==0: # even number of players
            # remove "even" players, including last
            playing = playing[0::2]
    return playing[0]

In [4]:
print(whiteElephantFast(3014387))

1834471


## Part 2

In [5]:
def whiteElephant2List(n):
    circle = [i for i in range(1,n+1)]
    while len(circle)>1:
        circle.pop(len(circle)//2)      # remove elf on the other side of the circle
        circle = circle[1:]+circle[0:1] # rotate circle (elf that just played goes to the end)
    return circle[0]

In [6]:
whiteElephant2List(5)

2

### Looking for patterns...

Brute force solution does not look promising for full input, especially with my poor-man list-based  implementation! I'll instead look for patterns...

In [7]:
for n in range(1,1000):
    if n-whiteElephant2List(n)==0:
        print(n,n-whiteElephant2List(n))

1 0
3 0
9 0
27 0
81 0
243 0
729 0


In [11]:
for n in range(9,82):
    print(n,whiteElephant2List(n))

9 9
10 1
11 2
12 3
13 4
14 5
15 6
16 7
17 8
18 9
19 11
20 13
21 15
22 17
23 19
24 21
25 23
26 25
27 27
28 1
29 2
30 3
31 4
32 5
33 6
34 7
35 8
36 9
37 10
38 11
39 12
40 13
41 14
42 15
43 16
44 17
45 18
46 19
47 20
48 21
49 22
50 23
51 24
52 25
53 26
54 27
55 29
56 31
57 33
58 35
59 37
60 39
61 41
62 43
63 45
64 47
65 49
66 51
67 53
68 55
69 57
70 59
71 61
72 63
73 65
74 67
75 69
76 71
77 73
78 75
79 77
80 79
81 81


## Patterns!

These are the patterns I found:

* Winner is last number n for n=3^m
* Winner is k for n=3^m+k for k=(1,2,...,3^(m+1))
* Winner is k for n=3^m+2k for k=(3^(m+1)+2,3^(m+1)+4,...,3^(m+2))

Implementing and checking them below...

In [12]:
def isPowerOfThree(n):
    # 1162261467 = 3^19
    return 1162261467 % n == 0; 

solution = 0
counter = 0
for n in range(9,82):
    if isPowerOfThree(n):
        solution = n
        power3 = n
        counter = 1
    else:
        if counter <= power3:
            solution = counter
            counter += 1
        else:
            solution = counter+1
            counter += 2
    print(n,whiteElephant2List(n),solution,whiteElephant2List(n)==solution)

9 9 9 True
10 1 1 True
11 2 2 True
12 3 3 True
13 4 4 True
14 5 5 True
15 6 6 True
16 7 7 True
17 8 8 True
18 9 9 True
19 11 11 True
20 13 13 True
21 15 15 True
22 17 17 True
23 19 19 True
24 21 21 True
25 23 23 True
26 25 25 True
27 27 27 True
28 1 1 True
29 2 2 True
30 3 3 True
31 4 4 True
32 5 5 True
33 6 6 True
34 7 7 True
35 8 8 True
36 9 9 True
37 10 10 True
38 11 11 True
39 12 12 True
40 13 13 True
41 14 14 True
42 15 15 True
43 16 16 True
44 17 17 True
45 18 18 True
46 19 19 True
47 20 20 True
48 21 21 True
49 22 22 True
50 23 23 True
51 24 24 True
52 25 25 True
53 26 26 True
54 27 27 True
55 29 29 True
56 31 31 True
57 33 33 True
58 35 35 True
59 37 37 True
60 39 39 True
61 41 41 True
62 43 43 True
63 45 45 True
64 47 47 True
65 49 49 True
66 51 51 True
67 53 53 True
68 55 55 True
69 57 57 True
70 59 59 True
71 61 61 True
72 63 63 True
73 65 65 True
74 67 67 True
75 69 69 True
76 71 71 True
77 73 73 True
78 75 75 True
79 77 77 True
80 79 79 True
81 81 81 True


### Part 2 iterative "pattern" solution

In [13]:
def isPowerOfThree(n):
    return 1162261467 % n == 0; 

def whiteElephant2Fast(N):
    # find closer power of 3 smaller then n
    for m in range(1,19):
        if 3**m <= N < 3**(m+1):
            break
    solution = 0
    counter = 0
    # find solution iterating on patterns!
    for n in range(3**m,N+1):
        if isPowerOfThree(n):
            solution = n
            power3 = n
            counter = 1
        else:
            if counter <= power3:
                solution = counter
                counter += 1
            else:
                solution = counter+1
                counter += 2
    return solution

In [14]:
whiteElephant2Fast(3014387)

1420064

### Part 2 mathematical "pattern" solution

In [15]:
def whiteElephant2Faster(N):
    for m in range(1,19):
        if 3**m <= N < 3**(m+1):
            break
    if N-3**m==0:
        return N
    if N-3**m < 3**m:
        return N-3**m 
    else:
        return 2*(N-3**m)-3**m

In [16]:
whiteElephant2Faster(3014387)

1420064